<a href="https://colab.research.google.com/github/andyliu14/PennApps2021F/blob/main/Drive_takeout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.137.0
    Uninstalling google-api-python-client-2.137.0:
      Successfully uninstalled google-api-python-client-2.137.0


In [2]:
from google.colab import auth
from googleapiclient.discovery import build

In [3]:
# Authenticate with Google Account
auth.authenticate_user()

# Create the Drive API client
drive_service = build('drive', 'v3')

In [54]:
def list_shared_files(service):
    """Lists all the names and IDs of the shared files and folders in your Google Drive."""
    page_token = None
    shared_files = []

    while True:
        try:
            # Query to list files and folders shared with you
            response = service.files().list(
                q="sharedWithMe=true",
                spaces='drive',
                fields='nextPageToken, files(id, name, mimeType)',
                pageToken=page_token
            ).execute()

            # Collect IDs, names, and mime types of the shared files and folders
            shared_files.extend(response.get('files', []))

            page_token = response.get('nextPageToken', None)
            if page_token is None:
                break
        except Exception as e:
            print(f"Error retrieving shared files: {e}")
            break

    return shared_files

def file_exists(service, file_name):
    """Check if a copy of the file already exists in the user's drive."""
    try:
        query = f"name = '{file_name}' and 'me' in owners"
        response = service.files().list(
            q=query,
            spaces='drive',
            fields="files(id, name)"
        ).execute()
        # Return True if any file with the specified name is found
        return len(response.get('files', [])) > 0
    except Exception as e:
        print(f"Error checking existence of {file_name}: {e}")
        return False

def copy_file(service, file_id, file_name, parent_folder_id):
    """Copies a file to the specified folder in the user's drive."""
    try:
        copied_file = {
            'name': f"Copy of {file_name}",
            'parents': [parent_folder_id]  # Place the file in the newly created folder
        }
        # Create a copy of the file in the specified folder
        service.files().copy(
            fileId=file_id,
            body=copied_file
        ).execute()
        print(f"Copied: {file_name} into folder ID: {parent_folder_id}")
    except Exception as e:
        print(f"Failed to copy {file_name}: {e}")

def copy_folder_recursively(service, folder_id, folder_name, parent_folder_id=None):
    """Recursively copies a folder and its contents to the user's drive, placing them in the correct parent folder."""
    try:
        # Create a copy of the folder in the user's drive, setting the parent folder if specified
        copied_folder = service.files().create(
            body={
                'name': f"Copy of {folder_name}",
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [parent_folder_id] if parent_folder_id else []
            }
        ).execute()
        print(f"Copied folder: {folder_name}")

        # List the contents of the folder
        page_token = None
        while True:
            response = service.files().list(
                q=f"'{folder_id}' in parents",
                fields='nextPageToken, files(id, name, mimeType)',
                pageToken=page_token
            ).execute()

            for item in response.get('files', []):
                # Recursively copy if it's a folder, otherwise copy as a file into the copied folder
                if item['mimeType'] == 'application/vnd.google-apps.folder':
                    copy_folder_recursively(service, item['id'], item['name'], copied_folder['id'])
                else:
                    copy_file(service, item['id'], item['name'], copied_folder['id'])

            page_token = response.get('nextPageToken', None)
            if page_token is None:
                break
    except Exception as e:
        print(f"Failed to copy folder {folder_name}: {e}")

def copy_shared_files(service, files):
    """Copies each shared file or folder to your Google Drive if a copy doesn't already exist."""
    for file in files:
        copy_name = f"Copy of {file.get('name', 'Unknown File')}"

        if 'id' not in file or 'name' not in file:
            print(f"Missing required fields for file: {file}")
            continue

        # Check if a copy of this file or folder already exists
        if file_exists(service, copy_name):
            print(f"Copy already exists: {copy_name}")
        else:
            if file['mimeType'] == 'application/vnd.google-apps.folder':
                # If it's a folder, copy it recursively
                copy_folder_recursively(service, file['id'], file['name'])
            else:
                # Otherwise, just copy the file
                copy_file(service, file['id'], file['name'], None)


In [39]:
shared_files = list_shared_files(drive_service)

In [42]:
new_copy_list = [file for file in shared_files if file['name'] in [ff['name'] for ff in failed_copies]]

In [50]:
new_copy_list = [ f for f in new_copy_list if f['name'].lower() != 'quant stuff']

In [55]:
failed, skipped = copy_shared_files(drive_service, new_copy_list)

Copy already exists: Copy of Final OIDD Project
Copy already exists: Copy of Group Cases
Copy already exists: Copy of 2020 lecture recordings
Copy already exists: Copy of CIS5200  STAFF (2022)
Copied folder: Final Project
Copied folder: CIS 520 for students (2022)
Copied folder: final exam
Copied: Comments on final solutions into folder ID: 1lSeQ1SbKWd8eYQkLvAw8YTuxNivmA_1G
Copied: CIS520_final_solns.pdf into folder ID: 1lSeQ1SbKWd8eYQkLvAw8YTuxNivmA_1G
Copied: CIS 520 Final project sources into folder ID: 1YPRrQDfaCo6m4-7co9h8qIZl0yM-QDSe
Copied: CIS 5200 | Pod Signup into folder ID: 1YPRrQDfaCo6m4-7co9h8qIZl0yM-QDSe
Copied folder: Homeworks
Copied folder: Final projects
Copied: checkpoint.tex into folder ID: 1aJo5I-TSj-YdfjirAdNZugdiVQNymrnV
Copied: checkpoint.pdf into folder ID: 1aJo5I-TSj-YdfjirAdNZugdiVQNymrnV
Copied folder: HW9
Copied: CIS520__2022_HW9SOLUTIONS.pdf into folder ID: 1mm5OBysnrO3pcWJamYuqfIHivRc-bxNI
Copied: CIS5200_HW9.pdf into folder ID: 1mm5OBysnrO3pcWJamYuqfIHiv

Failed to copy ESE215_Discovery2Intro.mp4: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/1J-_9LVxlaYmNA9V7MBWvp8jNfy9BeFGT/copy?alt=json returned "This file cannot be copied by the user.". Details: "[{'message': 'This file cannot be copied by the user.', 'domain': 'global', 'reason': 'cannotCopyFile'}]">
Copied folder: data
Copied: Test_Dst_NoAuction_DecPre_CF_7.txt into folder ID: 1fy98j225cYxk0QYfq295yPVV1aDMMK9b
Copied: Test_Dst_NoAuction_DecPre_CF_8.txt into folder ID: 1fy98j225cYxk0QYfq295yPVV1aDMMK9b
Copied: Test_Dst_NoAuction_DecPre_CF_9.txt into folder ID: 1fy98j225cYxk0QYfq295yPVV1aDMMK9b
Copied: Train_Dst_NoAuction_DecPre_CF_7.txt into folder ID: 1fy98j225cYxk0QYfq295yPVV1aDMMK9b
Copied folder: Practice Exams Fall 2016
Copied: Chem101Exam2PracQuesF16 into folder ID: 1-Z_z6KWDi0RqyCxc-cnx-hq5Gu45zLmQ
Copied: C101PE2AnsF16.pdf into folder ID: 1-Z_z6KWDi0RqyCxc-cnx-hq5Gu45zLmQ
Copied: Chem101Exam3PracQuesF16.pdf into folder ID: 1-Z_z6KWDi0RqyCxc-cnx-hq

TypeError: cannot unpack non-iterable NoneType object

In [51]:
len(n)

[{'mimeType': 'application/vnd.google-apps.folder',
  'id': '1bqHNu2A0x8_onVpqfR9pbwzcc0vZvEn7',
  'name': 'Final OIDD Project'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'id': '1wlx-dMpyMNkHMiKobi3AAYNtbxT-aZer',
  'name': 'Group Cases'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'id': '1hkmFEKKdrwfH4z4hIvCTXMPRA7uc6O2F',
  'name': '2020 lecture recordings'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'id': '1143k0TBwuTMFqpBGX7kKVEQzxk7-AMDU',
  'name': 'CIS5200  STAFF (2022)'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'id': '1H1csfFS7jqjEXNwvFIHAvdj4LMIlhRVI',
  'name': 'Final Project'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'id': '1tOnw-4kt1Fy0y1EaqwN1ZwE-yGtV-e9Q',
  'name': 'CIS 520 for students (2022)'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'id': '1a7y0lcJ5eAfBRLhQTKk6rT-REt_QJAu-',
  'name': 'Final Project'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'id': '1Mxm6Cd_RX1OVdP4j4Ycb0V4O5JPL